In [3]:
from bs4 import BeautifulSoup
import requests






url = "https://www.ime.usp.br/bioinfo/pos/teses-dissertacoes"
page = requests.get(url)

soup = BeautifulSoup(page.text, "html.parser")

In [4]:
list_of_thesis = soup.find(class_ = "striped-table")

all_links = list_of_thesis.find_all("a")

In [5]:
list_of_pre_thesis_urls = []

for link in all_links:
    link_now = link.get('href')
    if "teses" in link_now:
        list_of_pre_thesis_urls.append(link_now)

In [9]:
list_of_pre_thesis_urls[0:5]

['http://teses.usp.br/autor.php?autor=57A9A0FBF285',
 'http://teses.usp.br/autor.php?autor=17A7188B7471',
 'http://teses.usp.br/autor.php?autor=17A86D22C7A1',
 'http://teses.usp.br/autor.php?autor=B7AAB42FA53B',
 'http://teses.usp.br/autor.php?autor=97A5A5761BB9']

In [22]:
def get_thesis_url(pre_thesis_url):
    page = requests.get(pre_thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    box = soup.find(class_="dadosLinha dadosCor1")
    all_links = box.find_all("a")
    thesis_url = all_links[0].get("href")
    return(thesis_url)

def get_comittee_members (thesis_url):
    page = requests.get(thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    commitee = soup.select("#CorpoTexto > div:nth-child(23)")[0].text
    commitee = commitee.split("\n\n")
    student = soup.select("#CorpoTexto > div:nth-child(9)")[0].text
    students_commitee_dict = dict()
    students_commitee_dict[student] = commitee
    return(students_commitee_dict)


def get_abstracts(thesis_url):
    page = requests.get(thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    abstract = soup.select("#CorpoTexto > div:nth-child(35)")[0].text
    keywords = soup.select("#CorpoTexto > div:nth-child(33)")[0].text
    abstract_keywords = [abstract, keywords]
    student = soup.select("#CorpoTexto > div:nth-child(9)")[0].text
    students_abstracts_dict = dict()
    students_abstracts_dict[student] = abstract_keywords
    return(students_abstracts_dict)

In [13]:
bioinfo_commitees_dict = dict()

for pre_thesis_url in list_of_pre_thesis_urls:
    try:
        thesis_url = get_thesis_url(pre_thesis_url)
        commitee_dict = get_comittee_members(thesis_url)
        bioinfo_commitees_dict.update(commitee_dict)
    except:
        None

In [25]:
students_abstracts_dict = dict()

for pre_thesis_url in list_of_pre_thesis_urls:
    try:
        thesis_url = get_thesis_url(pre_thesis_url)
        abstracts_dict = get_abstracts(thesis_url)
        students_abstracts_dict.update(abstracts_dict)
    except:
        None


In [31]:
student_abstracts = pd.DataFrame.from_dict(students_abstracts_dict, orient="index")


student_abstracts.index.name = 'student'
student_abstracts.reset_index(inplace=True)

In [36]:
bioinfo_commitees_df = pd.read_csv("bioinfo_commitees_df_dirty.csv")

bioinfo_commitees_df.columns = ['student', 'na', 'president', 'comitee', 'comitee', 'comitee', 'comitee']

df = bioinfo_commitees_df.set_index('student')
df = df.stack()
df.index = df.index.rename('position', level=1)
df.name = "name"
df = df.reset_index()
df.head(10)

,student,position,name
0,Junier Marrero Gutierrez,president,"Koide, Tie"
1,Junier Marrero Gutierrez,comitee,"Martinez, Cristina Elisa Alvarez"
2,Junier Marrero Gutierrez,comitee,"Pinheiro, Daniel Guariz"
3,Junier Marrero Gutierrez,comitee,"Silva Neto, José Freire da\n"
4,Caio Rafael do Nascimento Santiago,president,"Digiampietri, Luciano Antonio"
5,Caio Rafael do Nascimento Santiago,comitee,"Chaim, Marcos Lordello"
6,Caio Rafael do Nascimento Santiago,comitee,"Ferreira, João Eduardo\n"
7,Suzane de Andrade Barboza,president,"Digiampietri, Luciano Antonio"
8,Suzane de Andrade Barboza,comitee,"Amaral, Fábio Sarubbi Raposo do"
9,Suzane de Andrade Barboza,comitee,"Lima, Ariane Machado"


In [37]:
def invert_names(name):
    name = name.replace("\n", "")
    try:
        bla = name.split(',')
        bla = bla[1] + " " + bla[0]
        return(bla)
    except:
        return(name)

professor_names = df['name']

df['name'] = professor_names.apply(invert_names)

In [38]:
tidy_bioinfo_comitees = df

In [39]:
tidy_bioinfo_comitees.to_csv("./tidy_bioinfo_comitees.csv")

In [42]:
tidy_bioinfo_comitees = tidy_bioinfo_comitees.merge(student_abstracts)

,student,position,name,0,1
0,Junier Marrero Gutierrez,president,Tie Koide,The family of metallo-β-lactamase enzymes comp...,\n\nasRNA\n\nBioinformatics\n\nHalobacterium s...
1,Junier Marrero Gutierrez,comitee,Cristina Elisa Alvarez Martinez,The family of metallo-β-lactamase enzymes comp...,\n\nasRNA\n\nBioinformatics\n\nHalobacterium s...
2,Junier Marrero Gutierrez,comitee,Daniel Guariz Pinheiro,The family of metallo-β-lactamase enzymes comp...,\n\nasRNA\n\nBioinformatics\n\nHalobacterium s...
3,Junier Marrero Gutierrez,comitee,José Freire da Silva Neto,The family of metallo-β-lactamase enzymes comp...,\n\nasRNA\n\nBioinformatics\n\nHalobacterium s...
4,Caio Rafael do Nascimento Santiago,president,Luciano Antonio Digiampietri,The computational needs of scientific experime...,\n\nDesktop grid\n\nPeer-to-Peer\n\nScheduling...
...,...,...,...,...,...
392,Fábio Nakano,president,Carlos Alberto de Braganca Pereira,This thesis presents a novel statistical model...,\n\nBayesian Networks\n\nDNA\n\nMicrossatelite...
393,Fábio Nakano,comitee,Jorge Alberto Achcar,This thesis presents a novel statistical model...,\n\nBayesian Networks\n\nDNA\n\nMicrossatelite...
394,Fábio Nakano,comitee,Fabio Gagliardi Cozman,This thesis presents a novel statistical model...,\n\nBayesian Networks\n\nDNA\n\nMicrossatelite...
395,Fábio Nakano,comitee,Louis Bernard Klaczko,This thesis presents a novel statistical model...,\n\nBayesian Networks\n\nDNA\n\nMicrossatelite...


In [43]:
tidy_bioinfo_comitees.to_csv("tidy_bioinfo_comitees.csv")